In [1]:
import keras
from keras import layers
import numpy as np

In [2]:
latent_dim=32
height=32
width=32
channels=3

## **Generator**

In [15]:
generator_input=keras.Input((latent_dim,))
x=layers.Dense(128*16*16)(generator_input)
x=layers.LeakyReLU()(x)
x=layers.Reshape((16,16,128))(x)

x=layers.Conv2D(265,5,padding='same')(x)
x=layers.LeakyReLU()(x)

x=layers.Conv2DTranspose(256,4,strides=2,padding='same')(x)
x=layers.LeakyReLU()(x)

x=layers.Conv2D(256,5,padding='same')(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(256,5,padding='same')(x)
x=layers.LeakyReLU()(x)

x=layers.Conv2D(channels,7,activation='tanh',padding='same')(x)
generator=keras.models.Model(generator_input,x)
generator.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)          │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 32768)               │       1,081,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_15 (LeakyReLU)           │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_2 (Reshape)                  │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 16, 16, 265)         │         848,265 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_16 (LeakyReLU)           │ (None, 16, 16, 265)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 32, 32, 256)         │       1,085,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_17 (LeakyReLU)           │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 32, 32, 256)         │       1,638,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_18 (LeakyReLU)           │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 32, 32, 256)         │       1,638,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_19 (LeakyReLU)           │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 32, 32, 3)           │          37,635 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,330,252 (24.15 MB)

 Trainable params: 6,330,252 (24.15 MB)

 Non-trainable params: 0 (0.00 B)

## **Discriminator**

In [12]:
discriminator_input = layers.Input(shape=(height, width, channels)) 
x = layers.Conv2D(128, 3)(discriminator_input) 
x = layers.LeakyReLU()(x) 
x = layers.Conv2D(128, 4, strides=2)(x) 
x = layers.LeakyReLU()(x) 
x = layers.Conv2D(128, 4, strides=2)(x) 
x = layers.LeakyReLU()(x) 
x = layers.Conv2D(128, 4, strides=2)(x) 
x = layers.LeakyReLU()(x) 
x = layers.Flatten()(x) 
x = layers.Dropout(0.4)(x) 
x = layers.Dense(1, activation='sigmoid')(x) 
discriminator = keras.models.Model(discriminator_input, x) 
discriminator.summary() 
discriminator_optimizer = keras.optimizers.RMSprop( 
 learning_rate=0.0008, 
 clipvalue=1.0, 
 decay=1e-8) 
discriminator.compile(optimizer=discriminator_optimizer, 
loss='binary_crossentropy')

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)           │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 30, 30, 128)         │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_11 (LeakyReLU)           │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 14, 14, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_12 (LeakyReLU)           │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 6, 6, 128)           │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_13 (LeakyReLU)           │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 2, 2, 128)           │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_14 (LeakyReLU)           │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 790,913 (3.02 MB)

 Trainable params: 790,913 (3.02 MB)

 Non-trainable params: 0 (0.00 B)

## **GANs**

In [17]:
discriminator.trainable=False
gan_input = keras.Input(shape=(latent_dim,)) 
gan_output = discriminator(generator(gan_input)) 
gan = keras.models.Model(gan_input, gan_output) 
gan_optimizer = keras.optimizers.RMSprop(learning_rate=0.0004, clipvalue=1.0, decay=1e-8) 
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

## Now,Let's train GANs Training

In [18]:
import os
from keras.preprocessing import image

In [19]:
(x_train,y_train),(_,_)=keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 271s 2us/step


In [23]:
x_train.shape

(50000, 32, 32, 3)

In [57]:
#choose Data With Class 6
# x_train = x_train[y_train.flatten() == 6] 
#normlizing Data
x_train = x_train.reshape(
(x_train.shape[0],) + 
(height, width, channels)).astype('float32') / 255. 
#Set HyberParameter
iterations = 10000 
batch_size = 20 
save_dir = r'D:\DLS\Generated Image' 
start = 0 

for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size, 
    latent_dim)) 
    generated_images = generator.predict(random_latent_vectors) 
    stop = start + batch_size 
    real_images = x_train[start: stop] 
    combined_images = np.concatenate([generated_images, real_images]) 
    labels = np.concatenate([np.ones((batch_size, 1)), 
    np.zeros((batch_size, 1))]) 
    labels += 0.05 * np.random.random(labels.shape) 
    d_loss = discriminator.train_on_batch(combined_images, labels) 
    random_latent_vectors = np.random.normal(size=(batch_size, 
    latent_dim)) 
    misleading_targets = np.zeros((batch_size, 1)) 
    a_loss = gan.train_on_batch(random_latent_vectors, 
    misleading_targets) 
    start += batch_size 
    if start > len(x_train) - batch_size: 
        start = 0 
    if step % 100 == 0: 
        gan.save_weights(r'D:\DLS\Generated Image\gan.weights.h5') 
        print('discriminator loss:', d_loss) 
        print('adversarial loss:', a_loss) 
        img = image.array_to_img(generated_images[0] * 255., scale=False) 
        img.save(os.path.join(save_dir, 
         'generated_frog' + str(step) + '.png')) 
        img = image.array_to_img(real_images[0] * 255., scale=False) 
        img.save(os.path.join(save_dir, 
         'real_frog' + str(step) + '.png'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
discriminator loss: 0.69426954
adversarial loss: [array(0.69426954, dtype=float32), array(0.69426954, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [55]:
x_train.shape

(5000, 32, 32, 3)